# PART 0: Initialize Authority Agent as an Issuing Authority

**Warning:** Run notebook only once after running `./manage start`. Before running it again, execute `./manage down` to delete the postgres databases. Otherwise, the storage will persist in the postgres database.

This notebook initializes the authority agent as an issuing authority on the Sovrin StagingNet. The purpose is to enable the authority to certify that manufacturer1, manufacturer2, manufacturer3 are manufacturers. The notebook consists of **??** steps:

1. Initialize authority agent and write DID to Sovrin StagingNet
2. Author schema to certify an agent is a city or a manufacturer (or use existing schema)
3. Store identifiers of Schema 1nd VC definition to access it in notebooks of other agents

In [1]:
%%javascript
document.title='Authority Agent'

<IPython.core.display.Javascript object>

#### Imports

In [2]:
from aries_cloudcontroller import AriesAgentController
import json
import libs.functions as fnc
import os
from pprintpp import pprint
from termcolor import colored

#### Variables

Set `issue_city_schema = True` and/or `issue_manufacturer_schema = True` if you want to issue a new (or update) Schema 1 and/or Schema 2

In [3]:
issue_city_schema = True
issue_manufacturer_schema = True

#### Init agent controller of Authority

In [4]:
api_key = os.getenv("ACAPY_ADMIN_API_KEY")
admin_url = os.getenv("ADMIN_URL")

agent_controller = AriesAgentController(admin_url,api_key)
print(f"Initialising a controller with admin api at {admin_url} and an api key of {api_key}")

Initialising a controller with admin api at http://authority-agent:3021 and an api key of adminApiKey


## 1 - Initialize Authority agent with a DID and scheme-authoring rights

Note: if defined a ACAPY_WALLET_SEED value for your agent then this function will return a DID, but this DID still needs to be written to the ledger. If you did not define a seed you will need to create a DID first.
### 1.1 - Get DID

In [5]:
did_obj = await fnc.get_public_did(agent_controller)
authority_did = did_obj["did"]

Successfully got public DID
{
    'did': 'XnKQj3iXVNpBBtFcArtxw2',
    'key_type': 'ed25519',
    'method': 'sov',
    'posture': 'posted',
    'verkey': 'Hn4QLxMFRXpAMCn91RoLuFLu1gEFbjPBDYhmdsmkQN2t',
}


### 1.2 - Write DID to Sovrin StagingNet

In [6]:
fnc.write_did_to_ledger(did_obj)

Status :  200
Reason :  NYM XnKQj3iXVNpBBtFcArtxw2 already exists on the ledger.


### 1.3 – Accept Transaction Author Agreement (TAA)

Although the Sovrin StagingNet is permissionless, before DID's have the authority to write to the ledger they must accept something called a transaction author agreement by signing it using the DID they have on the ledger.

As a global public ledger, the Sovrin Ledger and all its participants are subject to privacy and data protection regulations such as the EU General Data Protection Regulation (GDPR). These regulations require that the participants be explicit about responsibilities for Personal Data.

To clarify these responsibilities and provide protection for all parties, the Sovrin Governance Framework Working Group developed an agreement between Transaction Authors and the Sovrin Foundation. The TAA can be found at Sovrin.org. It ensures that users are aware of and consent to the fact that all data written to the Sovrin Ledger cannot be removed, even if the original author of the transaction requests its removal.

The TAA outlines the policies that users must follow when interacting with the Sovrin Ledger. When a user’s client software is preparing a transaction for submission to the network, it must include a demonstration that the user had the opportunity to review the current TAA and accept it. This is done by including some additional fields in the ledger write transaction: 

* A hash of the agreement
* A date when the agreement was accepted, and
* A string indicating the user interaction that was followed to obtain the acceptance.

The Indy client API used by Sovrin has been extended to allow users to review current and past agreements and to indicate acceptance through an approved user interaction pattern. - source: https://sovrin.org/preparing-for-the-sovrin-transaction-author-agreement/

For more details on TAA please read more at the following links:
* [Preparing for the Sovrin Transaction Author Agreement](https://sovrin.org/preparing-for-the-sovrin-transaction-author-agreement/)
* [How the recent approval of the Sovrin Governance Framework v2 affects Transaction Authors
](https://sovrin.org/how-the-recent-approval-of-the-sovrin-governance-framework-v2-affects-transaction-authors/)
* [TAA v2](https://github.com/sovrin-foundation/sovrin/blob/master/TAA/TAA.md)
* [TAA Acceptance Mechanism List (AML)](https://github.com/sovrin-foundation/sovrin/blob/master/TAA/AML.md)

In [7]:
await fnc.accept_taa_agreement(agent_controller)

Successfully signed TAA agreement


### 1.4 – Assign Authority with a public DID (if `ACAPY_WALLET_SEED` is not set in `agents/authority/.env`)

In [8]:
if did_obj["posture"] != "public":
    response = await agent_controller.wallet.assign_public_did(did_obj["did"])
    print(colored("Successfully intialised agent with Public DID: {d}".format(d=did_obj["did"]), "green", attrs=["bold"]))
else:
    print("Agent already has Public DID: {d}".format(d=did_obj["did"]))

Successfully intialised agent with Public DID: XnKQj3iXVNpBBtFcArtxw2


## 2 – Write VC schema to certify city-status (Schema 1) and manufacturer-status (Schema 2)


### 2.1 – Schema 1: VCs for Cities
#### 2.1.1 – Define schema

In [9]:
if issue_city_schema is True:
    # Define a unique schema name on the ledger, version the schema (to be able to update it), and define attributes in the schema
    schema_name = "certify-city-agency"
    schema_version = "0.0.1"
    attributes = ["city", "country", "isCityAgency"]

    # Write schema to ledger and await response
    schema_city_id = await fnc.write_schema(agent_controller, schema_name, schema_version, attributes)

else:
    # Else get stored variable
    schema_city_id = fnc.get_identifiers()["city_schema_identifiers"]["schema_id"]

Successfully wrote certify-city-agency schema:
{
    'schema': {
        'attrNames': ['isCityAgency', 'city', 'country'],
        'id': 'XnKQj3iXVNpBBtFcArtxw2:2:certify-city-agency:0.0.1',
        'name': 'certify-city-agency',
        'seqNo': 248532,
        'ver': '1.0',
        'version': '0.0.1',
    },
    'schema_id': 'XnKQj3iXVNpBBtFcArtxw2:2:certify-city-agency:0.0.1',
}


#### 2.1.2 – Author VC definition transaction to the public ledger
Author a verifiable credential definition transaction to the public ledger to specify the public cryptographic material the Authority agent uses to sign all VCs issued against Schema 1

In [10]:
# Tag and group specific credential definitions
tag = "default"

# Make Cred Def support revocation. Credentials issued using this definition will be able to be revoked.
# Note: ACAPY_TAILS_SERVER_BASE_URL env var must be properly configured
support_revocation = False

#TODO: is this where BBS+ needs to be specified?
cred_def_city_response = await agent_controller.definitions.write_cred_def(schema_city_id, tag, support_revocation)
cred_def_city_id = cred_def_city_response["credential_definition_id"]

#### 2.1.3 – Store identifiers in dictionary

In [11]:
city_identifiers = {"schema_id": schema_city_id, "cred_def": cred_def_city_id}

### 2.2 – Schema 2: VCs for Manufacturers
#### 2.2.1 – Define schema

In [12]:
if issue_manufacturer_schema is True:
    # Define a unique schema name on the ledger, version the schema (to be able to update it), and define attributes in the schema
    schema_name = "certify-manufacturer"
    schema_version = "0.0.1"
    attributes = ["manufacturerName", "manufacturerCountry", "manufacturerCity", "isManufacturer"]

    # Write schema to ledger and await response
    schema_manufacturer_id = await fnc.write_schema(agent_controller, schema_name, schema_version, attributes)
          
else:
    schema_manufacturer_id = fnc.get_identifiers()["manufacturer_schema_identifiers"]["schema_id"]

Successfully wrote certify-manufacturer schema:
{
    'schema': {
        'attrNames': [
            'manufacturerCountry',
            'manufacturerName',
            'isManufacturer',
            'manufacturerCity',
        ],
        'id': 'XnKQj3iXVNpBBtFcArtxw2:2:certify-manufacturer:0.0.1',
        'name': 'certify-manufacturer',
        'seqNo': 248534,
        'ver': '1.0',
        'version': '0.0.1',
    },
    'schema_id': 'XnKQj3iXVNpBBtFcArtxw2:2:certify-manufacturer:0.0.1',
}


#### 2.2.2 – Author VC definition transaction to the public ledger
Author a verifiable credential definition transaction to the public ledger to specify the public cryptographic material the Authority agent uses to sign all VCs issued against Schema 2

In [13]:
tag = "default" # Tag and group specific credential definitions
# Make Cred Def support revocation. Credentials issued using this definition will be able to be revoked.
# (Needs proper ACAPY_TRAILS_SERVER_BASE_URL env var configuration)
support_revocation = False

cred_def_manufacturer_response = await agent_controller.definitions.write_cred_def(schema_manufacturer_id, tag, support_revocation)
cred_def_manufacturer_id = cred_def_manufacturer_response["credential_definition_id"]

#### 2.2.3 – Store identifiers in dictionary

In [14]:
manufacturer_identifiers = {"schema_id": schema_manufacturer_id, "cred_def": cred_def_manufacturer_id}

## 3 – Store identifiers for use throughout other agents' notebooks

The IDs of the Schema 1nd VC definitions are required whenever the Authority agent issues credentials or constrains acceptable proof requests. For notebooks, it is easier to store the value pair as a string in a cell, and load them into the jupyter store. In real applications, the values should be stored in environment variables or a database.

Thus, the identifiers are printed and copied across the main business logic notebooks and stored as variables. This process only needs to be repeated when you ran `./manage.sh start` for the first time, or executed `./manage.sh down` (instead of `./manage.sh stop`).  

In [15]:
identifiers = {"city_schema_identifiers": city_identifiers, "manufacturer_schema_identifiers": manufacturer_identifiers, "authority_did": authority_did}
fnc.store_identifiers(dict(identifiers))

Successfully stored identifiers dictionary in libs/identifiers.json. (Will be needed in other notebooks and by other agents.)
{
    'authority_did': 'XnKQj3iXVNpBBtFcArtxw2',
    'city_schema_identifiers': {
        'cred_def': 'XnKQj3iXVNpBBtFcArtxw2:3:CL:248532:default',
        'schema_id': 'XnKQj3iXVNpBBtFcArtxw2:2:certify-city-agency:0.0.1',
    },
    'manufacturer_schema_identifiers': {
        'cred_def': 'XnKQj3iXVNpBBtFcArtxw2:3:CL:248534:default',
        'schema_id': 'XnKQj3iXVNpBBtFcArtxw2:2:certify-manufacturer:0.0.1',
    },
}


## 4 – Terminate Controller


In [16]:
await agent_controller.terminate()